<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/Autonomous_Agent_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Autonomus agent built using LangGraph

In [ ]:
!pip install uv
!pip install langchain langgraph langchain_community
!pip install python-dotenv

In [ ]:
import os
from typing import Dict , List , Tuple , Any , Optional , TypedDict , Annotated
#from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from google.colab import userdata

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

from langchain_community.chat_models import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub

# Initialize HuggingFaceHub with your Hugging Face API token and the model repo ID
# Set the OpenAI API key from Colab secrets
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")
llm = HuggingFaceHub(repo_id="meta-llama/Llama-3.1-8B-Instruct")

# Wrap it with ChatHuggingFace
llm = ChatHuggingFace(llm=llm)

#model_name = "meta-llama/Llama-2-7b-chat-hf" # Example: Replace with your desired Llama model
#model = AutoModelForCausalLM.from_pretrained(model_name)
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
#llm = HuggingFacePipeline(pipeline=pipe)

#llm = ChatOpenAI(model="meta-llama/Llama-3.1-8B-Instruct", temperature=0)
#define the agent state
class AgentState(TypedDict):
    """State of the agent"""
    question: str
    sub__questions: List[str]
    research_findings: List[str]
    current_sub_question: Optional[str]
    final_answer: Optional[str]
    evaluation: Optional[str]
    messages: List[Any]
#intialize the state graph
graph = StateGraph(AgentState)

In [ ]:
import os
from typing import Dict , List , Tuple , Any , Optional , TypedDict , Annotated
#from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from google.colab import userdata

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

from langchain_community.chat_models import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub

# Initialize HuggingFaceHub with your Hugging Face API token and the model repo ID
# Set the OpenAI API key from Colab secrets
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")
# llm = HuggingFaceHub(repo_id="meta-llama/Llama-3.1-8B-Instruct")

# Wrap it with ChatHuggingFace
#llm = ChatHuggingFace(llm=llm)



model_name = "meta-llama/Llama-3.1-8B-Instruct" # Example: Replace with your desired Llama model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipe)


#llm = ChatOpenAI(model="meta-llama/Llama-3.1-8B-Instruct", temperature=0)
#define the agent state
class AgentState(TypedDict):
    """State of the agent"""
    question: str
    sub_questions: List[str]
    research_findings: Dict[str, str]
    current_sub_question: Optional[str]
    final_answer: Optional[str]
    evaluation: Optional[str]
    messages: List[Any]
#intialize the state graph
graph = StateGraph(AgentState)

#define the breakdown node - divides the main question into sub-questions
def breakdown_question(state: AgentState) -> AgentState:
  """Breakdown the main question into sub-questions"""
  messages = [
      SystemMessage(content = "You are a research expert skilled at breaking complex questions into specific sub-questions. Your job is to analyze provided research questions and break it down into clear 3-5 sub-questions."),
      HumanMessage(content = f"Please break down the following research question into 3-5 sub questions: \n\n{state['question']}\n\n. Format your response as a list of sub-questions one per line , withour numbering or bullets")
      ]
  response = llm.invoke(messages)
  sub_questions = [q.strip() for q in response.split('\n') if q.strip()]
  return {
      ** state,
      "sub_questions":sub_questions,
      "research_findings":{},
      "messages":state["messages"] + [
          HumanMessage(content = f"Breaking down the question:{state['question']}"),
          AIMessage(content = f"I have broken down your question into these sub-questions:\n{response}")
          ]
      }
#define the research node - researches the current sub-question
def research_sub_question(state:AgentState) ->AgentState:
    """Research the current sub-question"""
    current_sub_question = state["sub_questions"][0] if not state ["current_sub_question"] else state["current_sub_question"]
    messages = [
      SystemMessage(content = "You are a thorough research assistant. Your taks is to provide adetailed answer to a specific research question using your knowledge"),
      HumanMessage(content = f"Please research the following question and provide a detailed answer: \n\n{current_sub_question}")
      ]
    response = llm.invoke(messages)
    research_findings = state["research_findings"].copy()
    research_findings[current_sub_question] = response

    return {
      ** state,
      "current_sub_question":current_sub_question,
      "research_findings":research_findings,
      "messages":state["messages"] + [
          HumanMessage(content = f"Researching:{current_sub_question}"),
          AIMessage(content = f"Research Findings:\n{response[:100]}...")
          ]
      }
#Define the next question selector - determines if we should research another question or synthesize
def select_next_steps(state:AgentState) ->str:
  """Determine whether to research the next question or move to synthesis"""
  researched_questions = set(state["research_findings"].keys())
  remanining_questions = [q for q in state["sub_questions"] if q not in researched_questions]
  if remanining_questions:
    state["current_sub_question"] = remanining_questions[0]
    return "research"
  else:
    return "synthesize"

#Define the synthesis node - combines research findings into a cohesive manner
def synthesize_findings(state:AgentState) ->AgentState:
  """Synthesize all research findings into a comprehensive answer"""
  findings_text = "\n".join([f"Sub-question: {q}:\n Findings: {f}" for q, f in state["research_findings"].items()])
  messages = [
         SystemMessage(content = "You are a research synthesis expert. Your task is to combine separate research findings into a cohesive,  well-structured answer to the original question."),
         HumanMessage(content =f"Original question:{state['question']}\n\n Research Findings:\n{findings_text}\n\n Please synthesize these findings into a comprehensive answer to the original question.")
         ]
  response = llm.invoke(messages)
  return{
         **state,
         "final_answer":response,
         "messages":state["messages"] +[
             HumanMessage(content="Synthesizing rsearch findings"),
             AIMessage(content=f"I 've synthesized the findings into a comprehensive answer:\n{response[:100]}...")
         ]
     }

#Define the evaluation node - assess the quality of the final answer
def evaluate_answer(state:AgentState) ->AgentState:
  """Evaluate the quality of the synthesized answer"""
  messages = [
         SystemMessage(content = "You are a critical evaluator of research answers. Your task is to identify any gaps , logical flows, or areas where the answer could be improved"),
         HumanMessage(content =f"Original question:{state['question']}\n\n Synthesized answer:{state['final_answer']}\n\n Please evauuate this answer , identifying any weakness or areas of improvement.")
         ]
  response = llm.invoke(messages)
  return{
         **state,
         "evaluation":response,
         "messages":state["messages"] +[
             HumanMessage(content="Evaluating the answer quality"),
             AIMessage(content=f"Evaluation:\n{response}")
         ]
     }
#define the refinement decision node - determines if the answer needs refinement
def needs_refinement(state:AgentState) ->str:
  """Determine if the answer needs refinement based on evaluation"""
  if not state["evaluation"] or "excellent" in state["evaluation"].lower() or "adequate" in state["evaluation"].lower():
    return "complete"
  else:
    return "refine"

#Define the evaluation node - assess the quality of the final answer
def refine_answer(state:AgentState) ->AgentState:
  """Refine the answer based on evaluation feedback"""
  messages = [
         SystemMessage(content = "You are a expert at refining research answers. Your task is to improve an answer based on specific evaluation feedback"),
         HumanMessage(content =f"Original question:{state['question']}\n\n Current answer:{state['final_answer']}\n\n Evaluation feedback:{state['evaluation']}\n\n Please provide an improved version of the answer that addresses the weakness identified in the evaluation.")
         ]
  response = llm.invoke(messages)
  return{
         **state,
         "final_answer":response,
         "messages":state["messages"] +[
             HumanMessage(content="Refining the answer based on evaluation..."),
             AIMessage(content=f"Refined answer :\n{response[:100]}...")
         ]
     }

In [ ]:
# Initialize the state graph again to avoid "Node already present" errors on re-execution
graph = StateGraph(AgentState)

#create the agent execution graph
graph.add_node("breakdown",breakdown_question)
graph.add_node("research",research_sub_question)
graph.add_node("synthesize",synthesize_findings)
graph.add_node("evaluate",evaluate_answer)
graph.add_node("refine",refine_answer)

#define the edges - the flow of execution between nodes
graph.add_edge("breakdown","research")
graph.add_conditional_edges(
    "research",
    select_next_steps,
    {
        "research": "research", #continue researching
        "synthesize":"synthesize" #move to Synthesis
    }
)
graph.add_edge("synthesize","evaluate")
graph.add_conditional_edges(
    "evaluate",
    needs_refinement,
    {
        "refine": "refine", #continue researching
        "complete":END #Finish execution
    }
)
graph.add_edge("refine","evaluate")
#set the entry point
graph.set_entry_point("breakdown")
#Compile the graph into an executable
research_agent = graph.compile()

#user interface
def run_research_agent(question:str):
  """Run the research agent on a given question"""
  initial_state={
      "question":question,
      "sub_questions":[],
      "research_findings":{},
      "current_sub_question":None,
      "final_answer":None,
      "evaluation":None,
      "messages":[]
  }
  print(f"Researching:{question}\n")
  print(f"Start the research process...\n")

  final_state = research_agent.invoke(initial_state)

  print("\n=== Sub-Questions===")
  for i , q in enumerate(final_state["sub_questions"],1):
    print(f"{i+1}.{q}")

  print("\n=== Final Answer===")
  print(final_state["final_answer"])

  print("\n=== Evaluation===")
  print(final_state["evaluation"])
  return final_state

#Example usage

if __name__ == "__main__":
  question = "What are the enviornmental and economic impacts of switching to renewable energy sources , what are the main challenges to widespread adoption?"
  run_research_agent(question)

In [ ]:
#creating memory and storing the conversation in the memory for retreival later....
!pip install langchain langchain_openai mem0ai python-dotenv

In [ ]:
import os
from typing import Literal , Dict , AnyStr , List
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage , HumanMessage , AIMessage
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
from mem0 import MemoryClient
from dotenv import load_dotenv

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from google.colab import userdata
# Set the OpenAI API key from Colab secrets

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["MEM0_API_KEY"] = userdata.get("MEM0_API_KEY")

load_dotenv()
#intialize llm
llm = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")
mem0=MemoryClient()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
#setup the coversation by setting up the prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are helpful AI Marketing Assistant. Use the provided context to persoanalize your responses and remember user preferences and past interactions
    Peronsalize the marketing plan generation and ask question from the use on specific campaigns that require user input. You can develop the marketing plan
    with the campaign information available to you in your context , campaign information such as Channel Name , Campaign Name , Campaign Efficiency , Campaign Spend
    Campaign ROI . Use also the simulated flat budget optimizations and other non flat budget optimizations . These optimizations are available via tool call to you"""),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessage(content="{input}")
])

In [ ]:
#define function for storing , retreiving chat into memory
def retrieve_context(query:str , user_id:str) ->List[Dict]:
  """retreive the conetext from Mem0"""
  try:
    memories = mem0.search(query , user_id = user_id)
    memory_list = memories["results"]
    serialized_memories = " ".join([mem[memories]] for mem in memory_list)
    chat_history = [
        {
            "role":"System",
            "content":f"Relevant Information{serialized_memories}"
        },
        {
            "role":"user",
            "content": query
        }
    ]
    return chat_history
  except Exception as e:
    print(f"Error Retrieving Memories:{e}")
    return [{"role":"user", "content":query}]


In [ ]:
def generate_response(input:str , context:List[Dict]) ->str:
  """ Generate a response using the lnaguage model"""
  chain = prompt | llm
  response = chain.invoke({"input":input , "chat_history":context})
  return response.content
def save_interaction(user_id:str, user_input:str , assistant_response:str):
  """ Save the interaction to Mem0"""
  #use exception handling in these scenarios to continue the flow of conversation
  try:
      interaction = [{
          "role":"user",
          "content":user_input
      },
      {
          "role":"assistant",
          "content":assistant_response
      }]
      result = mem0.add(interaction , user_id=user_id)
      print(f"Memory saved successfully:{len(result.get("results",[]))} memories added")
  except Exception as e:
      print(f"Error saving memory:{e}")


In [ ]:
#create chat function
def chat_turn(user_input:str, user_id:str) ->str:
  #Retrieve context
  context = retrieve_context(user_input , user_id)
  #Generate response
  assistant_response = generate_response(user_input , context)
  #Save interaction
  save_interaction(user_id , user_input , assistant_response)
  return assistant_response


In [ ]:
#main interaction function
if __name__ =="__main__":
  print("Welcome to Marketing Business Assistant . I can help generating marketing plans. How can I assist you today")
  user_id ="Nikhil"

  while True:
    user_input = input("You: ")
    if user_input in ["quit","exit","bye"]:
      print(f"Marketing Business Agent : Thanks for using my services. Have a great day ahead")
      break # break the conversation loop
    response = chat_turn(user_input , user_id)
    print(f"Marketing Business Agent: {response}")


In [ ]:
#code for ReACT Agent
%%capture
!pip install together
#if colab , download the data in current session
import os
if "COLAB_" in "".join(os.environ.keys()):
  !mkdir data
  !cd data ; wget https://raw.githubusercontent.com/togethercomputer/together-cookbook/refs/heads/main/Agents/DataScienceAgent/data/train_and_test2.csv

In [ ]:
#intialize Necessary Functions and Configurations
import os
import re
import json
import textwrap
import base64
from pathlib import Path
from typing import Dict, Optional , List , Any , Union

from together import Together
from IPython.display import Image , display

In [ ]:
#Agent configuration
#==================

#Model Selection
reasoning_model = "meta-llama/Llama-3.3-70B-Instruct-Turbo"  # primary model for reasoning and code generation
#xecution setting
max_iterations = 15 #number of max reasoning cycles
temperature = 0.2 #lower temperature for more consistent code generation
session_timeout = 3600 #session timeout in seconds

#display settings
max_output_length = 500
show_images = True
box_width = 80

#Intialize Together client
together = Together(api_key=os.environ["TOGETHER_API_KEY"])
code_interperter = together_client.code_interperter

In [ ]:
#function for running python code generated by the model - this uses CodeAct concept
def run_python(code:str,session_id:Optional[str] = None, files:Optional[List[Dict[str,str]]] = None):
  """ Executes Python Code
    Args: python code
    session_id :Optional session id to maintain state between execution
    files : Optional list for files to upload to the code interperter
  """
  """
  Returns : The output of the executed code as a JSON
  """
  try:
    kwargs = {"code":code , "language":"python"}
    if session_id:
      kwargs["session_id"] = session_id
    if files:
      kwargs["files"] = files
    response = code_interperter.run(**kwargs)
    result = {"session_id":response.data.session_id, "status":response.data.status,"outputs":[]}
    for output in response.data.outputs:
      result["outputs"].append({"type":output.type, "data":output.data})

    if response.data.errors:
      result["errors"] = response.data.errors
    return result
  except Exception as e:
    error_result = {"status":"error","error_message":str(e),"session_id":None}
    return error_result

In [ ]:
#LLM output processing function
def display_image(b64_image):
  decoded_image = base64.b64decode(b64_image)
  display(Image(data = decoded_image))
#function to parse the output of the output generated by model
def get_execution_summary(execution_result:Dict) ->str:
  """ execution"""
  if not execution_result:
    return "Execution failed - no result returned"

  #check execution status
  status = execution_result.get("status","unknown")
  summary_parts = [f"Execution Status:{status}"]
  #process outputs
  stdout_outputs = []
  display_outputs = []
  other_outputs = []

  if "outputs" in execution_result:
    for output in execution_result["outputs"]:
      output_type = output.get("type","unknown")
      output_data = output.get("data","")

      if output_type =="stdout":
        stdout_outputs.append(output_data)
      elif output_type =="display_data":
        if isinstance(output_data,dict):
          if "image/png" in output_data:
            display_outputs.append("Generated plot/image")
          if "text/plain" in output_data:
            display_outputs.append(f"Display:{output_data["text/plain"]}")
        else:
          display_outputs.append("Generated display output")
      else:
        other_outputs.append(f"{output_type}:{str(output_data)[:100]}")
  #add stdout outputs
  if stdout_outputs:
    summary_parts.append("\n".join(stdout_outputs))
  #add display outputs
  if display_outputs:
    summary_parts.append("\n".join(display_outputs))
  #add other outputs
  if other_outputs:
    summary_parts.append("\n".join(other_outputs))
  if "errors" in execution_result:
    summary_parts.append("\n".join(execution_result["errors"]))
  if not stdout_outputs and not display_outputs and not other_outputs and status =="success":
    summary_parts.append("Code executed successfully")
  return "\n".join(summary_parts)


In [ ]:
#ReACT Agent
class ReActDataScienceAgent():
  def __init__(self, client, session_id:Optional[str] = None, model:str = reasoning_model , max_iterations:int = max_iterations):
    self.client = client
    self.session_id = session_id
    self.model = model
    self.max_iterations = max_iterations
    self.system_prompt = """
    You are an expert Marketing Business Agent that follows a ReAct Framework(Reasoning + Acting)
    CRITICAL RULES:
    1.Execute ONLY ONE action at a time - this is non-negotiable
    2.Be methodical and detailed and deliberate in your approach
    3.For executing the action use the input as Media Channel , Campaign Name , Campaign Efficiency , Sales Generated by Campaign , ROI of Campaign , CAC of Campaign and Campaign Budget
    4.Using the above data always do two simulations flat budget simulation and increase the spend by +-5% and do simulation
    5.For flat budget simulation please use tool that is provided
    6.For Increase in budget and then do simulation use tool that is provided
    7.For Decrease in budget and then do simulation use tool that is provided
    8.You will be given external data such as Retail Inflation , Consumer Sentiment Index and other macro economic data that impact sales
    9.You will be given AGM reports and comp AGM reports as input use them pls and use the revenue section and forward looking section for strategy
    10.Once simulation results are done based on lowest CAC generate the marketing plan
    11.Never assume anything and ask before any other action

    IMPORTANT GUIDELINES:
    -Be explorative and creative , but cautious
    -Try things incrementally and observe the results
    -Never randomly guess
    -if you don't have access to simulation tool use "import os ; oslistdir() and check for Simulation.py"
    -When you see "Marketing Plan Generated Successfully" or "User says good with marketing plan" , it means your marketing plan worked
    You must strictly adhere to below format

    ##Format 1 - for taking action
    Thought - Reflect on what to do next. Analyze results from previous steps. Be descriptive and detailed about your reasoning, what you expect to see,
    how it builds on previous actions. Reference specific data points or patterns you have observed

    Action Input:
    '''python
    <simulation.py to run>
    '''
    ##Format 2 - ONLY when you have completely finished the task:
    Thought: Reflect and generate final marketing plan output
    Final Answer:
    [Provide a comprehensive marketing plan]

    ##Example of marketing plan
    Thought: I need to first get data for Media Channel , Campaign Name , Campaign Efficiency , Sales Generated by Campaign , ROI of Campaign ,
     CAC of Campaign and Campaign Budget. I need to use the external macro economic data and I need to run simulation continous simulation so that I am able
     to achieve lowest CAC and increase the sales and generate the plan

     Action Input:
     '''python
     import pandas as pd
     import Simulation as sim

     sim.simulation()
     '''

    """
    self.history = [{"role":"system","content":self.system_prompt}]

  def llm_call(self):
    response = self.client.chat.completions.create(
        model = self.model,
        messages = self.history,
        temperature = temperature,
        stream = False
    )
    return response.choices[0].message.content
  def parse_response(self):
    response = self.llm_call()


